# **Análisis y Comparación de Modelos de Machine Learning: Redes Neuronales**

En este proyecto se busca construir modelos de **Machine Learning** que permitan predecir el estado de aprobación de un préstamo utilizando los atributos de las personas solicitantes. Para ello, se analiza un conjunto de datos que contiene 14 atributos, entre ellos la edad, el ingreso anual, el puntaje crediticio, el propósito del préstamo, entre otros. La etiqueta de clase, loan_status, indica si el préstamo fue aprobado (1) o rechazado (0).

Utilizaremos la técnica de **Redes Neuronales** para abodar este problema, variando sus configuraciones y modificando hiperparámetros clave. El objetivo es comparar la exactitud (accuracy) de los modelos generados y determinar cuáles ofrecen mejores predicciones.

##### INTEGRANTES:
  1. Marcela Mazo Castro - 1843612
  2. Eyder Santiago Suárez Chávez - 2322714
  3. Erika García Muñoz - 2259395
  4. Juan José Moreno Jaramillo - 2310038

## Preparación de los Datos  
En esta sección, se cargan y preparan los datos para el entrenamiento y evaluación de los modelos.  

In [ ]:
# Importar bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Leer el archivo loan_data.csv
data = pd.read_csv("loan_data.csv")

# Dividir aleatoriamente los datos
train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)

# Normalización y codificación de los datos
numerical_features = ["person_age", "person_income", "person_emp_exp", "loan_amnt", "loan_int_rate", 
                      "loan_percent_income", "cb_person_cred_hist_length", "credit_score"]
categorical_features = ["person_gender", "person_education", "person_home_ownership", "loan_intent", 
                        "previous_loan_defaults_on_file"]

preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(), categorical_features)
])

## Modelos de Redes Neuronales  
En esta sección, se construyen y evalúan 5 modelos de redes neuronales variando la cantidad de capas ocultas, el número de neuronas por capa y los hiperparámetros principales como solver y activation. La implementación utiliza la clase MLPClassifier de scikit-learn para entrenar las redes neuronales con diferentes configuraciones topológicas y evaluar su desempeño en términos de exactitud.



In [5]:
# 1. Construcción de redes neuronales
topologies = [
    (50,), (100,), (50, 50), (100, 50), (100, 100)
]

results = []

for topology in topologies:
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', MLPClassifier(hidden_layer_sizes=topology, random_state=123, max_iter=500))
    ])
    
    # Entrenamiento
    model.fit(train_data.drop(columns="loan_status"), train_data["loan_status"])
    
    # Predicción
    predictions = model.predict(test_data.drop(columns="loan_status"))
    accuracy = accuracy_score(test_data["loan_status"], predictions)
    
    results.append({"Topology": topology, "Accuracy": accuracy})

### Análisis de Resultados  
Los resultados de las diferentes configuraciones topológicas se resumen en una tabla, mostrando la exactitud alcanzada por cada red neuronal.

### Estilos

In [16]:
class Colors:
    HEADER = '\033[95m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def print_pretty_table(df, enumerate_rows=False):
    if enumerate_rows:
        df = df.copy()
        df.insert(0, 'Red', range(0, len(df)))

    df_formatted = df.copy()
    for col in df_formatted.columns:
        if col != 'Alpha' and (df_formatted[col].dtype == 'float' or df_formatted[col].dtype == 'int'):
            def format_num(x):
                if isinstance(x, float):
                    return f"{x:.6f}"
                elif isinstance(x, int):
                    return str(x)
                else:
                    return str(x)
            df_formatted[col] = df_formatted[col].apply(format_num)

    columns = list(df_formatted.columns)
    col_widths = [
        max(len(str(item)) for item in [col] + df_formatted[col].astype(str).tolist()) + 2
        for col in columns
    ]
    top_left = '┌'
    top_right = '┐'
    bottom_left = '└'
    bottom_right = '┘'
    horizontal = '─'
    vertical = '│'
    top_sep = '┬'
    bottom_sep = '┴'
    mid_sep = '┼'
    left_sep = '├'
    right_sep = '┤'
    middle_sep = '┼'

    def create_border(left, sep, right):
        return left + sep.join([horizontal * width for width in col_widths]) + right

    def create_row(items, colored=False):
        if colored:
            return vertical + vertical.join([
                f"{Colors.BOLD + Colors.YELLOW}{str(item).center(width)}{Colors.ENDC}"
                for item, width in zip(items, col_widths)
            ]) + vertical
        else:
            return vertical + vertical.join([str(item).center(width) for item, width in zip(items, col_widths)]) + vertical

    print(create_border(top_left, horizontal, top_right))
    print(create_row(columns, colored=True))
    print(create_border(left_sep, mid_sep, right_sep))
    for i, row_data in df_formatted.iterrows():
        if i % 2 == 0:
            print(create_row(row_data, colored=False))
        else:
            row_str = vertical
            for item, width in zip(row_data, col_widths):
                row_str += f"\033[48;5;240m{str(item).center(width)}\033[0m{vertical}"
            print(row_str)
    print(create_border(bottom_left, horizontal, bottom_right))

### Tablas

In [20]:
#2 Resultados
results_df = pd.DataFrame(results)
results_df = results_df.head(5) 

print("Tabla de Resultados:")
print_pretty_table(results_df, enumerate_rows=True)

# 3. Hiperparámetro adicional
# Modificar un hiperparámetro adicional ¡Alpha!
print("\nResultados para diferentes valores de Alpha:")
alpha_results = []
alpha_variations = [0.0001, 0.01]

for alpha in alpha_variations:
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', MLPClassifier(hidden_layer_sizes=(100, 50), random_state=123, max_iter=500, alpha=alpha))
    ])
    model.fit(train_data.drop(columns="loan_status"), train_data["loan_status"])
    predictions = model.predict(test_data.drop(columns="loan_status"))
    accuracy = accuracy_score(test_data["loan_status"], predictions)
    alpha_results.append({'Alpha': alpha, 'Accuracy': accuracy})

alpha_df = pd.DataFrame(alpha_results)
print_pretty_table(alpha_df, enumerate_rows=False)

Tabla de Resultados:
┌─────────────────────────────┐
│ Red │  Topology  │ Accuracy │
├─────┼────────────┼──────────┤
│  0  │   (50,)    │ 0.923000 │
│  1  │   (100,)   │ 0.919222 │
│  2  │  (50, 50)  │ 0.906667 │
│  3  │ (100, 50)  │ 0.903111 │
│  4  │ (100, 100) │ 0.896667 │
└─────────────────────────────┘

Resultados para diferentes valores de Alpha:
┌───────────────────┐
│ Alpha  │ Accuracy │
├────────┼──────────┤
│ 0.0001 │ 0.903111 │
│  0.01  │ 0.911667 │
└───────────────────┘


## **Conclusiones**

En la tabla que se presentó previamente, se observa que la **red neuronal con la siguiente configuración de hiperparámetros** alcanza la mayor exactitud (accuracy):

- **Topología:** `(50,)`
- **Solver:** `adam` (Por defecto)
- **Función de activación:** `relu` (Por defecto)
- **Random State:** `123`

Esta configuración alcanzó una exactitud de aproximadamente **0.9230**. Por lo tanto, de las cinco topologías evaluadas, la red con **50 neuronas en una capa oculta** ofrece el mejor desempeño con los hiperparámetros actuales.



### **Análisis al variar el hiperparámetro `alpha`**

Se probaron dos valores de `alpha`: **0.0001** (valor por defecto) y **0.01**, manteniendo el resto de los hiperparámetros iguales a los de la mejor red encontrada. Los resultados son los siguientes:
- **alpha=0.0001:** Accuracy ≈ **0.9031**
- **alpha=0.01:** Accuracy ≈ **0.9117**

Observamos que al **aumentar el valor de `alpha`** de **0.0001** a **0.01**  se mejoró la exactitud del modelo de **0.9031** a **0.9117**, lo que significa que un mayor valor de regularización (`alpha`) ayudó a evitar el sobreajuste, logrando así una **ligera mejora** en el rendimiento del modelo.
